In [3]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [4]:
#DataFrames der Linken
df_linke_raw = pd.read_excel("../data/Parteien/@dieLinke_user_tweets_bearbeitet.xlsx")
df_linke_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
tweets_df_clean = df_linke_raw[['Name', 'original_tweets']]

In [2]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Create a new column with emji free text 
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

NameError: name 'tweets_df_clean' is not defined

In [6]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [7]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

möglich mittekoalition millionen einkommen hoch mieten hoch rente berlinerrunde


In [8]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [9]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,DIE LINKE,"Diejenigen, die tatsächlich verlieren werden, sind die, die mögliche ""Mitte-Koalition"", vergisst. Das sind die Millionen, deren Einkommen zu niedrig sind, die zu hohe Mieten zahlen oder die dringend eine höhere Rente brauchen. @SusanneHennig #BerlinerRunde #btw21","Diejenigen, die tatsächlich verlieren werden, sind die, die mögliche ""Mitte-Koalition"", vergisst. Das sind die Millionen, deren Einkommen zu niedrig sind, die zu hohe Mieten zahlen oder die dringend eine höhere Rente brauchen. @SusanneHennig #BerlinerRunde #btw21",diejenigen die tatsächlich verlieren werden sind die die mögliche mittekoalition vergisst das sind die millionen deren einkommen zu niedrig sind die zu hohe mieten zahlen oder die dringend eine höhere rente brauchen susannehennig berlinerrunde,möglich mittekoalition millionen einkommen hoch mieten hoch rente berlinerrunde
1,DIE LINKE,"Das ist kein schöner Abend. Das Ergebnis ist nicht in den letzten Wochen, sondern in den vergangenen Jahren entstanden, wo wir oft keine Klarheit gezeigt haben. Wir werden die nächsten Jahre im Bundestag nutzen, um uns weiter zu entwickeln. @SusanneHennig #btw21 #BerlinerRunde","Das ist kein schöner Abend. Das Ergebnis ist nicht in den letzten Wochen, sondern in den vergangenen Jahren entstanden, wo wir oft keine Klarheit gezeigt haben. Wir werden die nächsten Jahre im Bundestag nutzen, um uns weiter zu entwickeln. @SusanneHennig #btw21 #BerlinerRunde",das ist kein schöner abend das ergebnis ist nicht in den letzten wochen sondern in den vergangenen jahren entstanden wo wir oft keine klarheit gezeigt haben wir werden die nächsten jahre im bundestag nutzen um uns weiter zu entwickeln susannehennig berlinerrunde,schön abend ergebnis letzt wochen vergangen jahren klarheit nächst jahre bundestag
2,DIE LINKE,😊 👉🏼 https://t.co/OyifWhnc48 #jetzt https://t.co/ZxQ88XjVUi,https://t.co/OyifWhnc48 #jetzt https://t.co/ZxQ88XjVUi,jetzt,
3,DIE LINKE,"Vielen Dank allen Wähler*innen, die uns mit ihren Stimmen unterstützt haben. Vielen Dank liebe Genoss*innen und Sympathisant*innen für den großartigen Einsatz im Wahlkampf. \n#btw21 https://t.co/WIKvGLVoh6","Vielen Dank allen Wähler*innen, die uns mit ihren Stimmen unterstützt haben. Vielen Dank liebe Genoss*innen und Sympathisant*innen für den großartigen Einsatz im Wahlkampf. \n#btw21 https://t.co/WIKvGLVoh6",vielen dank allen wählerinnen die uns mit ihren stimmen unterstützt haben vielen dank liebe genossinnen und sympathisantinnen für den großartigen einsatz im wahlkampf \n,wählerinnen stimmen lieb genossinnen sympathisantinnen großartig einsatz wahlkampf
4,DIE LINKE,"@Janine_Wissler, @DietmarBartsch und @SusanneHennig zu den Ergebnissen der Bundestagswahl. #BTW21 #DIELINKE https://t.co/cNFH4nSXm1","@Janine_Wissler, @DietmarBartsch und @SusanneHennig zu den Ergebnissen der Bundestagswahl. #BTW21 #DIELINKE https://t.co/cNFH4nSXm1",janinewissler dietmarbartsch und susannehennig zu den ergebnissen der bundestagswahl dielinke,susannehennig ergebnissen bundestagswahl dielinke
...,...,...,...,...,...
413,DIE LINKE,Besser kann man die Wahl-Kampagne der #CDU nicht zusammen fassen. \n\n#DeutschlandGemeinsamMachen\n#CDUrausausderRegierung https://t.co/CT2ZXCGRME,Besser kann man die Wahl-Kampagne der #CDU nicht zusammen fassen. \n\n#DeutschlandGemeinsamMachen\n#CDUrausausderRegierung https://t.co/CT2ZXCGRME,besser kann man die wahlkampagne der cdu nicht zusammen fassen \n\ndeutschlandgemeinsammachen\ncdurausausderregierung,wahlkampagne cdurausausderregierung
414,DIE LINKE,"Wie berichtet wird, werben bei der #CDU Ehemann und Sohn der Parteisprecherin für ""bezahlbares #Wohnen"". Uns wundert es nicht, dass die #niewiederCDU keine echten Mieter*innen findet, die für sie werben. Warum? Darum 👇\n\n#DeutschlandGemeinsamMachen = #CDURausausderRegierung! https://t.co/dNOcAL3wre","Wie berichtet wird, werben bei der #CDU Ehemann und Sohn der Parteisprecherin für ""bezahlbare

In [10]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','janinewissler','link','joergschindler'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [12]:
unigram

,unigram,count
0,menschen,53
1,klimaschutz,36
2,euro,29
3,dielinke,27
4,politik,26
5,gut,20
6,einkommen,18
7,hoch,18
8,rente,17
9,soziale,17


In [13]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [14]:
bigram

,bigram,count
0,politik verloren,10
1,verloren zeit,10
2,schluss politik,8
3,soziale gerechtigkeit,7
4,mittler einkommen,7
5,euro monat,6
6,bundestagswahl dielinke,5
7,einkommen euro,5
8,nuklear teilhabe,5
9,hoch mindestlohn,4


In [15]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [16]:
trigram

,trigram,count
0,politik verloren zeit,10
1,schluss politik verloren,8
2,klein mittler einkommen,3
3,mittler einkommen euro,3
4,einkommen euro monat,3
5,zeit soziale gerechtigkeit,3
6,soziale gerechtigkeit klimaschutz,2
7,hoch mindestlohn gut,2
8,verloren zeit klimaschutz,2
9,deutsch wohnen co,2


In [17]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<418x529 sparse matrix of type '<class 'numpy.int64'>'
	with 1992 stored elements in Compressed Sparse Row format>

In [18]:
lda_model = LatentDirichletAllocation(
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=42,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [19]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['baerbock', 'schluss politik', 'politik', 'schluss', 'verloren', 'politik verloren', 'verloren zeit', 'grüne', 'zeit', 'klimaschutz']


Top 10 words for topic #1:
['gerecht', 'renten', 'susannehennig', 'thema', 'cdurausausderregierung', 'bildung', 'union', 'gut', 'uhr', 'spitzenkandidatin']


Top 10 words for topic #2:
['gut', 'schlussrunde', 'nuklear', 'bundestag', 'euro', 'bundestagswahl', 'rettenstattrüsten', 'bundesregierung', 'dielinke', 'menschen']


Top 10 words for topic #3:
['welt', 'geld', 'soziale gerechtigkeit', 'mietendeckel', 'gerechtigkeit', 'klimaschutz', 'soziale', 'politik', 'schlagabtausch', 'menschen']


Top 10 words for topic #4:
['groß', 'monat', 'mittler einkommen', 'mittler', 'dielinke', 'hoch', 'rente', 'einkommen', 'euro', 'menschen']




In [39]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  23.002800
0       51.163364 -159.968018       2        1  21.556719
2      -73.144554   21.523815       3        1  19.497012
3      -66.410133 -110.606438       4        1  19.108493
1       54.417324   12.443444       5        1  16.834977, topic_info=                    Term       Freq      Total Category  logprob  loglift
230          klimaschutz  29.000000  29.000000  Default  30.0000  30.0000
287             menschen  43.000000  43.000000  Default  29.0000  29.0000
378                rente  14.000000  14.000000  Default  28.0000  28.0000
145                 geld  10.000000  10.000000  Default  27.0000  27.0000
363      pressekonferenz   9.000000   9.000000  Default  26.0000  26.0000
..                   ...        ...        ...      ...      ...      ...
118                 euro   4.827649  24.019246   Topic5  -4.3120   0.1772
447        susannehennig   3.145883   7.056713   Topic5  -4.7403   0.9738
281  machtdaslandgerecht   2.859999   6.265946   Topic5  -4.8356   0.9974
426    spitzenkandidatin   2.702015  12.078592   Topic5  -4.8924   0.2843
455                thema   2.464766   6.288952   Topic5  -4.9843   0.8450

[292 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
4         2  0.251388                         abzug
4         4  0.754163                         abzug
6         4  0.900703                       aktuell
8         5  0.645278              alleinerziehende
9         5  0.645278  alleinerziehende verkäuferin
...     ...       ...                           ...
529       2  0.231522                          zeit
529       3  0.308696                          zeit
531       3  0.936155                  zeit soziale
537       1  0.250428                    ökologisch
537       4  0.751284                    ökologisch

[376 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 2])